In [1]:
import sklearn.metrics
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

%matplotlib
from matplotlib import pyplot as plt

from torch.utils.data import DataLoader
from torchinfo import summary
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing
import pysindy
from sklearn import svm

Using matplotlib backend: Qt5Agg


In [2]:
class Encoder(nn.Module):
    def __init__(self, x_dim, x_rom_dim):
        super(Encoder, self).__init__()
        h1_nodes = max(x_rom_dim, (x_dim + x_rom_dim) // 2)     # max(3, 23//2) = 11
        h2_nodes = max(x_rom_dim, (x_dim + x_rom_dim) // 2)     # max(3, 23//2) = 11
        h3_nodes = max(x_rom_dim, (x_dim + x_rom_dim) // 4)     # max(3, 23//4) = 5
        self.input = nn.Linear(x_dim, h1_nodes)
        self.h1 = nn.Linear(h1_nodes, h2_nodes)
        self.h2 = nn.Linear(h2_nodes, h3_nodes)
        self.h3 = nn.Linear(h3_nodes, x_rom_dim)
        nn.init.kaiming_uniform_(self.input.weight)
        nn.init.kaiming_uniform_(self.h1.weight)
        nn.init.kaiming_uniform_(self.h2.weight)
        nn.init.kaiming_uniform_(self.h3.weight)
    def forward(self, x_in):
        xe1 = F.elu(self.input(x_in))
        xe2 = F.elu(self.h1(xe1))
        xe3 = F.elu(self.h2(xe2))
        x_rom_out = (self.h3(xe3))

        # xe1 = F.leaky_relu(self.input(x_in))
        # xe2 = F.leaky_relu(self.h1(xe1))
        # xe3 = F.leaky_relu(self.h2(xe2))
        # x_rom_out = (self.h3(xe3))
        return x_rom_out

In [3]:
class Decoder(nn.Module):
    def __init__(self, x_dim, x_rom_dim):
        super(Decoder, self).__init__()
        h1_nodes = max(x_rom_dim, (x_dim + x_rom_dim) // 4)     # max(3, 23//4) = 5
        h2_nodes = max(x_rom_dim, (x_dim + x_rom_dim) // 2)     # max(3, 23//2) = 11
        h3_nodes = max(x_rom_dim, (x_dim + x_rom_dim) // 2)     # max(3, 23//2) = 11
        self.input = nn.Linear(x_rom_dim, h1_nodes)
        self.h1 = nn.Linear(h1_nodes, h2_nodes)
        self.h2 = nn.Linear(h2_nodes, h3_nodes)
        self.h3 = nn.Linear(h3_nodes, x_dim)
        nn.init.kaiming_uniform_(self.input.weight)
        nn.init.kaiming_uniform_(self.h1.weight)
        nn.init.kaiming_uniform_(self.h2.weight)
        nn.init.kaiming_uniform_(self.h3.weight)
    def forward(self, x_rom_in):
        xe1 = F.elu(self.input(x_rom_in))
        xe2 = F.elu(self.h1(xe1))
        xe3 = F.elu(self.h2(xe2))
        x_full_out = (self.h3(xe3))

        # xe1 = F.leaky_relu(self.input(x_rom_in))
        # xe2 = F.leaky_relu(self.h1(xe1))
        # xe3 = F.leaky_relu(self.h2(xe2))
        # x_full_out = (self.h3(xe3))
        return x_full_out

In [4]:
class Autoencoder:
    def __init__(self, x_dim, x_rom_dim=3):
        self.encoder = Encoder(x_dim, x_rom_dim)
        self.decoder = Decoder(x_dim, x_rom_dim)
        self.scaler_x = preprocessing.MinMaxScaler()
    def process_and_normalize_data(self, dataframe):
        x = []
        for i in range(20):
            x.append(dataframe["x{}".format(i)].to_numpy(dtype=np.float32))

        # x is fit with n rows and 20 columns, so we need to reshape it to this for transform
        # Tranpose x to obtain a 2D array with shape (num_trajectories * time, 20)
        x = np.array(x).transpose()
        self.scaler_x.fit(x)
        x = self.scaler_x.transform(x)
        x_tensor = torch.tensor(x)
        return x_tensor
    def transform_data_without_fit(self, dataframe):
        x = []
        for i in range(20):
            x.append(dataframe["x{}".format(i)].to_numpy(dtype=np.float32))

        # x is fit with n rows and 20 columns, so we need to reshape it to this for transform
        # Tranpose x to obtain a 2D array with shape (num_trajectories * time, 20)
        x = np.array(x).transpose()
        x = self.scaler_x.transform(x)
        x_tensor = torch.tensor(x)
        return x_tensor
    def fit(self, dataframe, test_dataframe):
        x = self.process_and_normalize_data(dataframe)
        x_test = self.transform_data_without_fit(test_dataframe)
        self.encoder.train()
        self.decoder.train()
        mb_loader = torch.utils.data.DataLoader(x, batch_size=100, shuffle=True)
        param_wrapper = nn.ParameterList()
        param_wrapper.extend(self.encoder.parameters())
        param_wrapper.extend(self.decoder.parameters())
        optimizer = optim.SGD(param_wrapper, lr=0.01)
        criterion = nn.MSELoss()
        for epoch in range(2000):
            for x_mb in mb_loader:
                optimizer.zero_grad()
                x_rom_mb = self.encoder(x_mb)
                x_full_pred_mb = self.decoder(x_rom_mb)
                loss = criterion(x_full_pred_mb, x_mb)
                loss.backward()
                optimizer.step()

            # Test on the test dataset at this epoch
            self.encoder.eval()
            self.decoder.eval()
            with torch.no_grad():
                x_rom = self.encoder(x_test)
                x_full_pred = self.decoder(x_rom)
                loss = criterion(x_full_pred, x_test)
                mae = metrics.mean_absolute_error(x_full_pred, x_test)
                mape = metrics.mean_absolute_percentage_error(x_full_pred, x_test)
                print("Held out test dataset - Epoch {}: MSE = {}, MAE = {}, MAPE = {}".format(epoch, loss, mae, mape))
            self.encoder.train()
            self.decoder.train()
    def encode(self, dataframe):
        print(dataframe)
        x = self.transform_data_without_fit(dataframe)
        self.encoder.eval()
        with torch.no_grad():
            x_rom = self.encoder(x)
        x_rom = x_rom.numpy()
        # x_rom = self.scaler_x.inverse_transform(x_rom)
        x_rom_df_cols = []
        for i in range(x_rom.shape[1]):
            x_rom_df_cols.append("x{}_rom".format(i))
        x_rom_df = pd.DataFrame(x_rom, columns=x_rom_df_cols)
        print(x_rom_df)
        return x_rom_df
    def decode(self, x_rom_nparr):
        # Expected shape of x_rom_nparr is (x_rom_dim, )
        # Reshape to match decoder dimensions
        x_rom_nparr = x_rom_nparr.reshape(1, 3)
        self.decoder.eval()
        with torch.no_grad():
            x_decoded = self.decoder(x_rom_nparr)

        # Scale x_decoded into x_full
        x_decoded = self.scaler_x.inverse_transform(x_decoded)

        # Return x_decoded as a numpy array, not pandas dataframe (to the basinhopper)
        return x_decoded

In [5]:
def load_pickle(filename):
    with open(filename, "rb") as model:
        pickled_autoencoder = pickle.load(model)
    print("Pickle loaded: " + filename)
    return pickled_autoencoder

In [ ]:
def generate_data_for_svm():
    autoencoder = load_pickle("heatEq_autoencoder_3dim_lr001_batch100_epoch2000.pickle")
    num_samples = 3000

    # Rough calculation: 263 to 313 = 50, 263 to 343 = 80, 80*20 = 1600
    # Generate PASS data: x5 is less than 313
    pass_states = []
    for i in range(num_samples):
        # x0 to x19 can be anything in the range of 263 to 343 (from initial-10 to setpoint+10)
        x0_x4 = np.random.randint(low=263, high=343, size=(5, ))
        # x5 must be less than 313
        x5 = np.random.randint(low=263, high=313)
        x6_x19 = np.random.randint(low=263, high=343, size=(14,))
        # Get one pass state, append to list of passed state - label is 1
        state = np.hstack((x0_x4, x5, x6_x19)).flatten()
        pass_states.append(state)

    # Convert the passed states in the full state space to the reduced space
    pass_states = np.array(pass_states)
    df_cols = []
    for i in range(20):
        df_cols.append("x{}".format(i))
    pass_states_df = pd.DataFrame(pass_states, columns=df_cols)
    print(pass_states_df)
    rom_pass_states = autoencoder.encode(pass_states_df)

    # Create a vector of ones to label the pass state
    ones_vector = np.ones(shape=num_samples, dtype=int)
    ones_vector_df = pd.DataFrame(ones_vector, columns=["pass"])
    # hstack pass dataframe with label
    pass_df = pd.concat([rom_pass_states, ones_vector_df], axis=1)

    # Generate FAIL data: x5 is more than 313
    fail_states = []
    for i in range(num_samples):
        # x0 to x19 can be anything in the range of 263 to 343 (from initial-10 to setpoint+10)
        x0_x4 = np.random.randint(low=263, high=343, size=(5, ))
        # x5 fails if its more than 313
        x5 = np.random.randint(low=314, high=333)
        x6_x19 = np.random.randint(low=263, high=343, size=(14,))
        # Get one fail state, append to list of failed states - label is 0
        state = np.hstack((x0_x4, x5, x6_x19)).flatten()
        fail_states.append(state)

    # Convert the failed states in the full state space to the reduced space
    fail_states = np.array(fail_states)
    #
    fail_states_df = pd.DataFrame(fail_states, columns=df_cols)
    print(fail_states_df)
    rom_fail_states = autoencoder.encode(fail_states_df)

    # Create a vector of zeros to label the fail state
    zeros_vector = np.zeros(shape=num_samples, dtype=int)
    zeros_vector_df = pd.DataFrame(zeros_vector, columns=["pass"])
    # hstack pass dataframe with label
    fail_df = pd.concat([rom_fail_states, zeros_vector_df], axis=1)

    # vstack the SVM training data and save to csv
    svm_training_data = pd.concat([pass_df, fail_df], axis=0)
    svm_training_data.to_csv("svm_training_data.csv")

In [6]:
def run_svm():
    training_df = pd.read_csv("svm_training_data.csv")
    x_y = training_df.to_numpy()
    X = x_y[:, 1:4]
    print(X)
    Y = x_y[:, -1]
    print(Y)
    model = svm.SVC(kernel='linear', verbose=1)
    clf = model.fit(X, Y)
    # The equation of the separating plane is given by all x so that np.dot(svc.coef_[0], x) + b = 0.
    # Solve for w3 (z)
    z = lambda x, y: (-clf.intercept_[0] - clf.coef_[0][0] * x - clf.coef_[0][1] * y) / clf.coef_[0][2]
    tmp = np.linspace(-1.5, 2.5, 40)
    x, y = np.meshgrid(tmp, tmp)
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot3D(X[Y == 0, 0], X[Y == 0, 1], X[Y == 0, 2], 'ob')
    ax.plot3D(X[Y == 1, 0], X[Y == 1, 1], X[Y == 1, 2], 'sr')
    ax.plot_surface(x, y, z(x, y), alpha=0.2)
    ax.set_xlim3d(-1, 2)
    ax.set_ylim3d(-1, 2)
    ax.set_zlim3d(-1, 2)
    ax.view_init(0, -60)
    plt.ion()
    plt.show()
    plt.savefig("svm_decision_boundary.svg", format="svg")

In [7]:
if __name__ == "__main__":
    # generate_data_for_svm()
    run_svm()

[[ 1.0195172  -0.46792153 -0.05973433]
 [ 1.0361204  -0.45224002  0.7710551 ]
 [ 1.2189639  -0.77077734 -0.10829385]
 ...
 [ 1.0265098  -0.5322219   0.03354688]
 [ 0.71588176 -0.365802    0.18486194]
 [ 1.2546597  -0.8108769  -0.08369084]]
[1. 1. 1. ... 0. 0. 0.]
[LibSVM]